In [12]:
import os
import cv2
import numpy as np
import openslide
# from distutils.util import strtobool
from tqdm.autonotebook import tqdm
import pandas as pd
# from jupyfuncs.glob import makedirs

In [13]:
os.getcwd()

'/Users/hulu/projects/bioinfo'

In [3]:
def makedirs(path: str, isfile: bool = False) -> None:
    """Creates a directory given a path to either a directory or file.
    If a directory is provided, creates that directory. If a file is provided (i.e. :code:`isfile == True`),
    creates the parent directory for that file.
    Args:
        path (str): Path to a directory or file. 
        isfile (bool, optional): Whether the provided path is a directory or file.Defaults to False.
    """    
    if isfile:
        path = os.path.dirname(path)
    if path != '':
        os.makedirs(path, exist_ok=True)

In [14]:
wsi_name = '/Users/hulu/Downloads/browser/BCL2A1-483a.qptiff'
txt_name = './data/BCL2A1.txt'
tmaspotsize = 8500
outdir = '/Users/hulu/Downloads/browser/BCL2A1/'
level = 0
wsi_filename = wsi_name
txt_filename = txt_name
tmaspot_size = tmaspotsize
makedirs(outdir)


In [15]:
print(f"Extracting patches for {wsi_filename} into {outdir}")
# -

# if not os.path.isdir(f"{outdir}"):
#     os.mkdir(f"{outdir}")

slide = openslide.OpenSlide(wsi_filename)

# Print the slide's downsample info
level_dims = slide.level_dimensions[level]
level_downsample = slide.level_downsamples[level]
print(f'Downsample at level {level} is: {level_downsample}')
print(f'WSI dimensions at level {level} are: {level_dims}.')

bounds_x = float(slide.properties['openslide.bounds-x']) \
    if ("openslide.bounds-x") in slide.properties.keys() \
    else 100
bounds_y = float(slide.properties['openslide.bounds-y']) \
    if ("openslide.bounds-y") in slide.properties.keys() \
    else 180
ratio_x = 3.98
ratio_y = 3.98
# ratio_x = 1.0/float(slide.properties['openslide.mpp-x'])
# ratio_y = 1.0/float(slide.properties['openslide.mpp-y'])

# dataset = np.loadtxt(txt_filename, dtype=str, delimiter='\t', skiprows=1)
dataset = np.array(pd.read_csv(txt_filename, sep='\t'))

Extracting patches for /Users/hulu/Downloads/browser/BCL2A1-483a.qptiff into /Users/hulu/Downloads/browser/BCL2A1/
Downsample at level 0 is: 1.0
WSI dimensions at level 0 are: (72960, 106560).


In [16]:
print(f"Number of rows in txt file ：{len(dataset)}") 
i = 0
for row in tqdm(dataset):
    fname, label, missing, x, y = row
    if not missing:
        x = (float(x) * ratio_x) + bounds_x
        y = (float(y) * ratio_y) + bounds_y
        print(f"Extracting spot {label} at location", (x, y))
        scaled_spot_size = (
            int(tmaspot_size / level_downsample),
            int(tmaspot_size / level_downsample)
        )
        tmaspot = np.asarray(
            slide.read_region(
                (int(x - tmaspot_size * 0.5), int(y - tmaspot_size * 0.5)),
                level,
                (8500, 8500)
            )
        )[:, :, 0:3]
        tmaspot = cv2.cvtColor(tmaspot, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f"{outdir}/{label}.png", tmaspot)
    else:
        print(f'The spot {label} is missing, skipping!')
    i += 1
    if i >= 2:
        break
print('Extracted all the spots!')

Number of rows in txt file ：80


  0%|          | 0/80 [00:00<?, ?it/s]

Extracting spot A-1 at location (11338.724, 6940.03)
Extracting spot A-2 at location (20501.878, 6748.592000000001)
Extracting spot A-3 at location (29160.368000000002, 6051.6939999999995)
Extracting spot A-4 at location (38814.256, 6051.6939999999995)
Extracting spot A-5 at location (48268.746, 6848.092000000001)
Extracting spot A-6 at location (57424.736000000004, 6748.592000000001)
Extracting spot A-7 at location (66481.22600000001, 7644.092000000001)
Extracting spot A-8 at location (75139.716, 7843.092000000001)
Extracting spot B-1 at location (10947.887999999999, 16103.582)
Extracting spot B-2 at location (20203.378, 16302.582)
Extracting spot B-3 at location (29359.368000000002, 16302.582)
Extracting spot B-4 at location (38515.358, 15705.582)
Extracting spot B-5 at location (47970.246, 16402.082000000002)
Extracting spot B-6 at location (57225.736000000004, 16004.082)
Extracting spot B-7 at location (66381.72600000001, 16999.082)
Extracting spot B-8 at location (75139.716, 17098

KeyboardInterrupt: 